In [1]:
import pandas as pd

In [2]:
# Load population data
population_data = pd.read_csv("plz_einwohner.csv")

# Load charging stations data
charging_stations_data = pd.read_excel("Ladesaeulenregister_SEP.xlsx")

# Check the data structure
# print(population_data.head())
# print(charging_stations_data.head())

In [3]:
population_data.head()

,plz,note,einwohner,qkm,lat,lon
0,1067,01067 Dresden,11957,6.866839,51.06019,13.71117
1,1069,01069 Dresden,25483,5.339213,51.03964,13.73030
2,1097,01097 Dresden,14821,3.298022,51.06945,13.73781
3,1099,01099 Dresden,28018,58.505818,51.09272,13.82842
4,1108,01108 Dresden,5876,16.447222,51.15180,13.79227


In [4]:
print(population_data.columns)

Index(['plz', 'note', 'einwohner', 'qkm', 'lat', 'lon'], dtype='object')


In [5]:
population_data.rename(columns={"plz": "PLZ", "einwohner": "Population"}, inplace=True)

In [6]:
population_data.head(3)

,PLZ,note,Population,qkm,lat,lon
0,1067,01067 Dresden,11957,6.866839,51.06019,13.71117
1,1069,01069 Dresden,25483,5.339213,51.03964,13.73030
2,1097,01097 Dresden,14821,3.298022,51.06945,13.73781


In [7]:
charging_stations_data.head()

,Betreiber,Anzeigename (Karte),Straße,Hausnummer,Adresszusatz,Postleitzahl,Ort,Kreis/kreisfreie Stadt,Bundesland,Breitengrad,...,Public Key3,Steckertypen4,P4 [kW],Public Key4,Steckertypen5,P5 [kW],Public Key5,Steckertypen6,P6 [kW],Public Key6
0,Albwerk GmbH & Co. KG,Albwerk GmbH & Co. KG,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Landkreis Alb-Donau-Kreis,Baden-Württemberg,"48,442398",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,smopi® - Multi Chargepoint Solution GmbH,smopi,Albstraße,15,NaN,72535,Heroldstatt,Landkreis Alb-Donau-Kreis,Baden-Württemberg,"48,449424",...,NaN,AC Typ 2 Steckdose,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,smopi® - Multi Chargepoint Solution GmbH,smopi,Albstraße,15,NaN,72535,Heroldstatt,Landkreis Alb-Donau-Kreis,Baden-Württemberg,"48,449424",...,NaN,AC Typ 2 Steckdose,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,smopi® - Multi Chargepoint Solution GmbH,smopi,Albstraße,15,NaN,72535,Heroldstatt,Landkreis Alb-Donau-Kreis,Baden-Württemberg,"48,449424",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Albwerk GmbH & Co. KG,Albwerk GmbH & Co. KG,Parkplatz Campingplatz,0,NaN,72589,Westerheim,Landkreis Alb-Donau-Kreis,Baden-Württemberg,"48,5105",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = charging_stations_data.loc[:,['Postleitzahl', 'Bundesland', 'Breitengrad', 'Längengrad', 'Nennleistung Ladeeinrichtung [kW]']]

In [9]:
df.head(3)

,Postleitzahl,Bundesland,Breitengrad,Längengrad,Nennleistung Ladeeinrichtung [kW]
0,72535,Baden-Württemberg,"48,442398","9,659075",22.0
1,72535,Baden-Württemberg,"48,449424","9,672201",22.0
2,72535,Baden-Württemberg,"48,449424","9,672201",22.0


In [10]:
df.rename(columns  = {"Nennleistung Ladeeinrichtung [kW]":"KW", "Postleitzahl": "PLZ"}, inplace = True)

In [11]:
df.head(2)

,PLZ,Bundesland,Breitengrad,Längengrad,KW
0,72535,Baden-Württemberg,"48,442398","9,659075",22.0
1,72535,Baden-Württemberg,"48,449424","9,672201",22.0


In [12]:
df['Breitengrad'] = df['Breitengrad'].astype('str')
df['Längengrad'] = df['Längengrad'].astype('str')

In [13]:
df['Breitengrad'] = df['Breitengrad'].str.replace(',','.')
df['Längengrad'] = df['Längengrad'].str.replace(',','.')

In [14]:
charging_counts = df.groupby("PLZ").size().reset_index(name='ChargingStations')

In [15]:
charging_counts

,PLZ,ChargingStations
0,1011,1
1,1036,1
2,1062,1
3,1067,60
4,1068,1
...,...,...
6897,99986,2
6898,99988,2
6899,99991,1
6900,99994,2


In [16]:
merged_data = pd.merge(population_data, charging_counts, on="PLZ", how="left")

In [17]:
population_data.shape

(8170, 6)

In [18]:
charging_counts.shape

(6902, 2)

In [19]:
merged_data.shape

(8170, 7)

In [20]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8170 entries, 0 to 8169
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PLZ               8170 non-null   int64  
 1   note              8170 non-null   object 
 2   Population        8170 non-null   int64  
 3   qkm               8170 non-null   float64
 4   lat               8170 non-null   float64
 5   lon               8170 non-null   float64
 6   ChargingStations  6606 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 446.9+ KB


In [21]:
merged_data["ChargingStations"].fillna(0, inplace=True)

C:\Users\Gaurav\AppData\Local\Temp\ipykernel_6340\450564457.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data["ChargingStations"].fillna(0, inplace=True)


In [22]:
merged_data["PopulationPerStation"] = merged_data["Population"] / (merged_data["ChargingStations"] + 1)

In [23]:
merged_data.head(2)

,PLZ,note,Population,qkm,lat,lon,ChargingStations,PopulationPerStation
0,1067,01067 Dresden,11957,6.866839,51.06019,13.71117,60.0,196.016393
1,1069,01069 Dresden,25483,5.339213,51.03964,13.73030,51.0,490.057692


In [24]:
merged_data.drop(['note', 'qkm'], axis= 1, inplace= True)

In [25]:
merged_data.head(2)

,PLZ,Population,lat,lon,ChargingStations,PopulationPerStation
0,1067,11957,51.06019,13.71117,60.0,196.016393
1,1069,25483,51.03964,13.73030,51.0,490.057692


In [26]:
merged_data.rename(columns= {"PLZ": "plz", "Population": "Einwohner", "lat": "Breitengrad", "lon": "Längengrad"}, inplace= True)
merged_data.head(2)

,plz,Einwohner,Breitengrad,Längengrad,ChargingStations,PopulationPerStation
0,1067,11957,51.06019,13.71117,60.0,196.016393
1,1069,25483,51.03964,13.73030,51.0,490.057692


In [53]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191 entries, 621 to 811
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   plz                   191 non-null    int64  
 1   Einwohner             191 non-null    int64  
 2   Breitengrad           191 non-null    float64
 3   Längengrad            191 non-null    float64
 4   ChargingStations      191 non-null    float64
 5   PopulationPerStation  191 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 10.4 KB


In [52]:
merged_data = merged_data[(merged_data.plz > 10000) & (merged_data.plz < 14200)]

In [54]:
merged_data.to_csv("processed_data.csv", index=False)

In [29]:
import folium
from folium.plugins import HeatMap

In [30]:
geojson_file = "plz-5stellig.geojson"

In [31]:
import geopandas as gpd

In [32]:
geojson_file

'plz-5stellig.geojson'

In [33]:
import geopandas as gpd

In [34]:
geo_data = gpd.read_file(geojson_file)

In [35]:
geo_data.head(2)

,plz,note,einwohner,qkm,geometry
0,64743,"Situation unklar, evtl. haben die Häuser Marba...",3,0.082066,"POLYGON ((8.98124 49.60761, 8.9814 49.6073, 8...."
1,81248,81248 München,121,1.984763,"POLYGON ((11.39468 48.14729, 11.39484 48.14654..."


In [68]:
geo_data[geo_data.plz == 10115]

,plz,note,einwohner,qkm,geometry
3838,10115,10115 Berlin Mitte,20313,2.393948,"POLYGON ((13.36522 52.53628, 13.3657 52.53582,..."


In [36]:
geo_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 8170 entries, 0 to 8169
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   plz        8170 non-null   object  
 1   note       8170 non-null   object  
 2   einwohner  8170 non-null   int32   
 3   qkm        8170 non-null   float64 
 4   geometry   8170 non-null   geometry
dtypes: float64(1), geometry(1), int32(1), object(2)
memory usage: 287.4+ KB


In [37]:
geo_data['plz'] = geo_data['plz'].astype(int)

In [38]:
geo_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 8170 entries, 0 to 8169
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   plz        8170 non-null   int64   
 1   note       8170 non-null   object  
 2   einwohner  8170 non-null   int32   
 3   qkm        8170 non-null   float64 
 4   geometry   8170 non-null   geometry
dtypes: float64(1), geometry(1), int32(1), int64(1), object(1)
memory usage: 287.4+ KB


In [55]:
geo_merged = geo_data.merge(merged_data, on="plz", how="left")

In [69]:
geo_merged = geo_merged[(geo_merged.plz > 10000) & (geo_merged.plz < 14200)]

In [70]:
geo_merged.head(3)

,plz,note,einwohner,qkm,geometry,Einwohner,Breitengrad,Längengrad,ChargingStations,PopulationPerStation
3446,10789,10789 Berlin Schöneberg,4131,0.448729,"POLYGON ((13.33073 52.49955, 13.33114 52.49952...",4131.0,52.50215,13.33801,10.0,375.545455
3476,10409,10409 Berlin Prenzlauer Berg,21656,1.751824,"POLYGON ((13.42671 52.54456, 13.42724 52.54435...",21656.0,52.54423,13.44208,12.0,1665.846154
3528,10319,10319 Berlin,22951,3.574872,"POLYGON ((13.49279 52.49743, 13.49399 52.49667...",22951.0,52.50056,13.52299,5.0,3825.166667


In [77]:
geo_merged.to_csv("processed1_data.csv", index=False)

In [78]:
geo_merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 191 entries, 3446 to 8047
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   plz                   191 non-null    int64   
 1   note                  191 non-null    object  
 2   einwohner             191 non-null    int32   
 3   qkm                   191 non-null    float64 
 4   geometry              191 non-null    geometry
 5   Einwohner             191 non-null    float64 
 6   Breitengrad           191 non-null    float64 
 7   Längengrad            191 non-null    float64 
 8   ChargingStations      191 non-null    float64 
 9   PopulationPerStation  191 non-null    float64 
dtypes: float64(6), geometry(1), int32(1), int64(1), object(1)
memory usage: 15.7+ KB


In [71]:
geo_merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 191 entries, 3446 to 8047
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   plz                   191 non-null    int64   
 1   note                  191 non-null    object  
 2   einwohner             191 non-null    int32   
 3   qkm                   191 non-null    float64 
 4   geometry              191 non-null    geometry
 5   Einwohner             191 non-null    float64 
 6   Breitengrad           191 non-null    float64 
 7   Längengrad            191 non-null    float64 
 8   ChargingStations      191 non-null    float64 
 9   PopulationPerStation  191 non-null    float64 
dtypes: float64(6), geometry(1), int32(1), int64(1), object(1)
memory usage: 15.7+ KB


In [74]:
m = folium.Map(location=[52.52, 13.40], zoom_start=11)

In [75]:
folium.Choropleth(
    geo_data=geo_merged,
    data=geo_merged,
    columns=["plz", "PopulationPerStation"],
    key_on="feature.properties.plz",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Population per Charging Station"
).add_to(m)

In [76]:
m.save("berlin_charging_stations_map.html")

In [60]:
import streamlit as st
import pandas as pd
import geopandas as gpd
import folium
from streamlit_folium import st_folium

2024-11-29 11:23:08.970 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [57]:
data = pd.read_csv("processed_data.csv")

In [62]:
def create_map():
    m = folium.Map(location=[52.5200, 13.4050], zoom_start=8)
    folium.Choropleth(
        geo_data=geo_merged,
        data=geo_merged,
        columns=["plz", "PopulationPerStation"],
        key_on="feature.properties.plz",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Population per Charging Station"
    ).add_to(m)
    return m

In [61]:
st.title("Berlin Electric Charging Station Demand")

st.write("This app visualizes demand for electric vehicle charging stations in Berlin.")

map_ = create_map()
st_folium(map_, width=700, height=500)

2024-11-29 11:23:11.854 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 11:23:12.202 
  command:

    streamlit run d:\ase\.venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-11-29 11:23:12.218 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 11:23:12.218 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 11:23:12.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 11:23:12.230 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-29 11:23:12.233 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


KeyboardInterrupt: 

In [ ]:
streamlit run main.py